In [2]:
import numpy as np
import tensorflow.contrib.distributions as ds
from sklearn.utils import shuffle
from math import pi
import tensorflow as tf
import argparse
import ram

In [32]:
sess = tf.InteractiveSession()

In [11]:
def make_dna_seq(length):
    one_hot_bases = np.eye(4)
    sample_indices = np.random.randint(0, 4, length)
    return one_hot_bases[sample_indices]


In [17]:
def make_atac_seq(length):
    return np.random.randint(0, 20, length).reshape([length, 1])
    

In [90]:
dna = make_dna_seq(100)
atac = make_atac_seq(100)

In [91]:
dna.shape, atac.shape

((100, 4), (100, 1))

In [92]:
input_ = np.hstack([dna, atac])

In [93]:
input_

array([[  1.,   0.,   0.,   0.,   7.],
       [  0.,   0.,   0.,   1.,  15.],
       [  1.,   0.,   0.,   0.,   9.],
       [  0.,   1.,   0.,   0.,  11.],
       [  0.,   0.,   1.,   0.,   8.],
       [  0.,   0.,   1.,   0.,   7.],
       [  0.,   1.,   0.,   0.,   6.],
       [  1.,   0.,   0.,   0.,  17.],
       [  0.,   0.,   1.,   0.,  16.],
       [  0.,   0.,   0.,   1.,   1.],
       [  0.,   0.,   1.,   0.,  17.],
       [  0.,   0.,   1.,   0.,  19.],
       [  1.,   0.,   0.,   0.,  10.],
       [  0.,   0.,   0.,   1.,   2.],
       [  0.,   0.,   0.,   1.,  17.],
       [  0.,   0.,   0.,   1.,  15.],
       [  0.,   0.,   1.,   0.,  19.],
       [  1.,   0.,   0.,   0.,   8.],
       [  1.,   0.,   0.,   0.,   2.],
       [  0.,   0.,   0.,   1.,  13.],
       [  0.,   0.,   1.,   0.,   0.],
       [  0.,   1.,   0.,   0.,   1.],
       [  0.,   0.,   1.,   0.,   6.],
       [  0.,   1.,   0.,   0.,  12.],
       [  1.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   1.,

In [94]:
def get_glimpses(data, num_resolutions):
    glimpses = []
    for i in range(num_resolutions):
        resolution = 2**i
        glimpse = tf.nn.pool(
            input=data,
            window_shape=[resolution],
            strides=[resolution],
            pooling_type='MAX',
            padding='SAME')
        glimpses.append(glimpse)
    return glimpses

In [99]:
glimpse_size = 4
num_resolutions = 4
glimpses = get_glimpses(np.expand_dims(atac, 0), num_resolutions)

In [112]:
def index_glimpses(num_resolutions, glimpses, location, glimpse_size):
    to_concatenate = []
    for i in range(num_resolutions):
        glimpse = glimpses[i]
        location_index = int(location / 2**i)
        
        left_pad = max(glimpse_size - location_index, 0)
        right_pad = max(glimpse_size + location_index - glimpse.get_shape().as_list()[1], 0)
        
        left_index = max(location_index - glimpse_size, 0)
        right_index = min(location_index + glimpse_size, glimpse.get_shape().as_list()[1])
        
        resolution = glimpse[:, left_index: right_index, :]
        resolution = tf.pad(resolution, [[0, 0], [left_pad, right_pad], [0, 0]])
        
        to_concatenate.append(resolution)
    return tf.concat(to_concatenate, axis=-1)


In [113]:
index_glimpses(num_resolutions, glimpses, 50, glimpse_size)

<tf.Tensor 'concat_5:0' shape=(1, 8, 4) dtype=int64>

In [102]:
glimpses

[<tf.Tensor 'max_pool_47/Squeeze:0' shape=(1, 100, 1) dtype=int64>,
 <tf.Tensor 'max_pool_48/Squeeze:0' shape=(1, 50, 1) dtype=int64>,
 <tf.Tensor 'max_pool_49/Squeeze:0' shape=(1, 25, 1) dtype=int64>,
 <tf.Tensor 'max_pool_50/Squeeze:0' shape=(1, 13, 1) dtype=int64>]